In [1]:
import pandas as pd
import urllib.request
import warnings
warnings.filterwarnings('ignore')

In [2]:
#read in courts and judges
court_list = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/courts.csv")
judges = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/judges.csv")
#dictionary for replacing the court id with the court name
court_code = dict(court_list[["Court ID", "Court Name"]].values)

In [3]:
#Replace codes in the dataframe with their actual meaning
#Ex. replace "W" with "White" for defendant race
def map_values(df):
    
    df = df.rename(columns = {'Charge Code':'Case Type'})
    #Fill in NaN values 
    df['Sentence Y'] = df['Sentence Y'].fillna(0)
    df['Sentence M'] = df['Sentence M'].fillna(0)
    df['Sentence D'] = df['Sentence D'].fillna(0)
    df['Probation D'] = df['Probation D'].fillna(0)
    df['Probation Y'] = df['Probation Y'].fillna(0)
    df['Probation M'] = df['Probation M'].fillna(0)
    df["Charge Class"] = df['Charge Class'].fillna("Un")
    df['Judge'] = df['Judge'].fillna("N/A") #Nans are filled with unknowns 
    
    f = open("/Users/hinaljajal/Downloads/State-Sentencing-Project/CSV Processing/edited_codes.csv")
    for line in f:
        line = line.strip('\n')
        line = line.split(",")
        df = df.replace({line[2]:{line[0]:line[1]}})
    f.close()
    
    df['Race'] = df['Race'].fillna("UNKNOWN")
    return df

In [4]:
def case_matcher(cases, judges_court):
    """
    cases: a dataframe of cases for 1 specifc court
    judges_court: list of judges for specific court
    """
    cases["Year"]=cases["Last Hearing Date"].str[-4:]

    #Merges the columns by the judge and the hearing year
    cases['Year']=cases['Year'].astype(int)
    cases['Judge']=cases['Judge'].astype(str)
    judges_court = judges_court.drop(['Year'], axis = 1)
    judges_court['Judge']=judges_court['Judge'].astype(str)

    cases = cases.merge(judges_court, how="left", on = ["Judge"])
    
    #dropping the name and second court name column
    cases=cases.drop(['Name','Court Name'], axis=1)
    
    #Saves the dataframe to a csv
#     file_name = f'/Users/hinaljajal/Downloads/{court}.csv'
#     cases.to_csv(file_name, index = False)
    
    return cases

In [7]:
import sqlite3
conn = sqlite3.connect("cases.db") # create a database in current directory called cases.db
separate_court_data = []
for court in court_list["Court ID"]:
    data = pd.read_csv(f"/Users/hinaljajal/Downloads/FinishedCourts/{court}.csv")
    data = map_values(data)
    #Slices the judges for the particular court 
    judges_court = judges[judges["Court Name"]==court_code[court]]
    #replace court code with name, ie "001C" with "Accomack"
    data["Court"] = court_code[court]
    data = case_matcher(data, judges_court)
    data.to_sql("final_cases", conn, if_exists = "append", chunksize = 100000, index = False)
    separate_court_data.append(data) #add to list for concatenation
    
final_cases = pd.concat(separate_court_data, ignore_index = True)
conn.close()

In [6]:
import sqlite3
conn = sqlite3.connect("cases.db")# create a database in current directory called cases.db
final_cases.to_sql("final_cases", conn, if_exists = "replace"index = False)
conn.close()